In [1]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [2]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "./Dataset Korosi/"

In [3]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=32,class_mode='categorical',target_size=(150,150))   
val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, subset='validation', batch_size=32,class_mode='categorical',target_size=(150,150))   

Found 304 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [4]:
mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(150,150,3),
    weights='imagenet',
    include_top=False,
)

In [5]:
model_mobilenet = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer( input_shape=(150,150,3)),
    mobilenet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])


In [6]:
import matplotlib.pyplot as plt

In [7]:
f = open("mobilenet.csv", "w")
f.write("epochs,loss,val_loss,acc,val_acc")
f.close()


In [8]:
for times in range(10):
    epo = (times+1)*10
    tf.keras.backend.clear_session()
    model_mobilenet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_mobilenet.fit(train_generator,validation_data=val_generator,epochs=epo)
    
    f = open("mobilenet.csv", "a")
    f.write("\n"+str(epo)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./mobilenet/loss_'+str(epo)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./mobilenet/acc_'+str(epo)+'.png')
    
    plt.clf()

Epoch 1/10
2/2 [==============================] - 14s 6s/step - loss: 14.7352 - acc: 0.4000 - val_loss: 4.8442 - val_acc: 0.7500
Epoch 2/10
2/2 [==============================] - 6s 4s/step - loss: 0.7619 - acc: 0.8167 - val_loss: 4.1731 - val_acc: 0.5333
Epoch 3/10
2/2 [==============================] - 6s 5s/step - loss: 2.9590 - acc: 0.6167 - val_loss: 8.2710 - val_acc: 0.7500
Epoch 4/10
2/2 [==============================] - 6s 4s/step - loss: 0.5688 - acc: 0.9333 - val_loss: 19.3829 - val_acc: 0.6333
Epoch 5/10
2/2 [==============================] - 6s 4s/step - loss: 2.0393 - acc: 0.8667 - val_loss: 14.4262 - val_acc: 0.7000
Epoch 6/10
2/2 [==============================] - 6s 4s/step - loss: 0.3716 - acc: 0.9333 - val_loss: 10.9641 - val_acc: 0.7833
Epoch 7/10
2/2 [==============================] - 6s 5s/step - loss: 0.1877 - acc: 0.9333 - val_loss: 7.8086 - val_acc: 0.8333
Epoch 8/10
2/2 [==============================] - 6s 5s/step - loss: 0.1698 - acc: 0.9667 - val_loss: 6.01

<Figure size 432x288 with 0 Axes>